In [1]:
import logging

import numpy as np
import pandas as pd
import geopandas as gpd
import pyarrow.compute as pc
from matplotlib import pyplot as plt
import pyarrow.dataset as ds
from shapely.geometry import Point, Polygon, MultiPolygon
from scipy.spatial import KDTree

from src.datalake import Datalake

logging.getLogger('matplotlib.font_manager').setLevel(logging.ERROR)
plt.style.use('bmh')


def initialize_figure(w: int = 14, h: int = 10) -> any:
    return plt.subplots(figsize=(w, h))

In [2]:
data = Datalake("../data")
state_geoms = data.query_states()

scope_states = ["California"]
scope_multipoly = state_geoms[state_geoms.name.isin(scope_states)].geometry.iloc[0]
stations = data.query_stations()
stations = stations[stations.geometry.within(scope_multipoly)]
stations["num_samples"] = 0

In [4]:
dataset = ds.dataset(data.datalake_root / data.paths["ghcnd_clean_daily"])
for i, (row_id, s) in enumerate(stations.iterrows()):
    n = dataset.filter(pc.field("station_id") == s.station_id).to_table().num_rows
    stations.loc[row_id, "num_samples"] = n
    print(i, n)

0 360
1 0
2 366
3 352
4 29
5 0
6 0
7 0
8 366
9 0
10 0
11 0
12 36
13 0
14 0
15 28
16 355
17 363
18 0
19 0
20 0
21 303


KeyboardInterrupt: 

In [7]:
dataset.schema

station_id: string
year: int32
month: int32
day: int32
prcp: double
snow: double
snwd: double
tmax: double
tmin: double
awnd: double
elevation: double
geometry: binary
  -- field metadata --
  ARROW:extension:name: 'geoarrow.wkb'
  ARROW:extension:metadata: '{"crs": "{\"$schema\":\"https://proj.org/sch' + 1587
-- schema metadata --
pandas: '{"index_columns": [{"kind": "range", "name": null, "start": 0, "' + 1587
geo: '{"primary_column": "geometry", "columns": {"geometry": {"encoding":' + 1362